In [17]:
import numpy as np
from math import ceil, sqrt
from cvxpy import *
import cvx

In [39]:
m = 10
n = 5
numpy.random.seed(1)
A = numpy.random.randn(m, n)
b = numpy.random.randn(m, 1)

# Construct the problem.
x = Variable(n)
objective = Minimize(np.ones(n) * x)
constraints = [A * x <= b]
prob = Problem(objective, constraints)

In [41]:
prob.__dict__

{'_cached_data': {'CBC': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x109de4cf8>,
  'CVXOPT': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x109de4cc0>,
  'ECOS': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x11c3aa748>,
  'ECOS_BB': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x11c3aa6d8>,
  'ELEMENTAL': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x109de45f8>,
  'GLPK': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x109de4e48>,
  'GLPK_MI': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x11c3aa978>,
  'GUROBI': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x109de4da0>,
  'JULIA_OPT': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x11c3aa710>,
  'LS': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x109de4fd0>,
  'MOSEK': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x11c3aa400>,
  'SCS': <cvxpy.problems.problem_data.problem_data.ProblemData at 0x11c3aa9

# Linear Programming

In [4]:
# Parameters

K = 5 # integer, noise dim
n = 10  # integer, x dim
m = 7 # integer, number of constraints

P = np.random.rand(n, K) # Matrix n x K shape of ellipsoidal uncertainty
A = np.random.rand(n, m) # Matrix n x m
B = np.random.rand(m) # Vector m

In [22]:
def oracle(U, A, B, eps):
    return np.linalg.pinv((A + P.dot(U.T)).T).dot(B)

In [6]:
def update_u(u_prev, x, P, eta):
    vec = u_prev + eta * P.T.dot(x)
    return vec / max(np.linalg.norm(vec, 2), 1)

In [30]:
def dual_subgradient_robust(eps, D, G):
    T = ceil(G ** 2 * D ** 2 / eps ** 2)
    eta =  D / (G * sqrt(T))
    U = np.random.rand(m, K)
    U_prev = np.random.rand(K)
    X = np.zeros((T, n))
    X_prev = np.random.rand(n)
    for t in range(T):
        for i in range(m):
            U_prev = update_u(U_prev, X_prev, P, eta)
            U[i] = U_prev
        X_prev = oracle(U, A, B, eps)
        if X_prev is None:
            return None
        else:
            X[t] = X_prev
    return np.mean(X, axis=0)

In [31]:
eps = 1e-1
D = 2
G = 1

res = dual_subgradient_robust(eps, D, G)

In [33]:
res

array([-0.20558111,  0.01904341,  0.19525488, -0.41798284, -0.22132591,
       -0.19843838,  0.36747816,  0.3727173 , -0.35025577,  0.46855103])